In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [3]:
df = pd.read_csv("data/24-25.csv")

In [4]:
df.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA
0,F1,16/08/2024,19:45,Le Havre,Paris SG,1,4,A,0,1,...,1.93,2.00,1.95,1.97,1.95,2.02,1.89,1.95,1.95,2.02
1,F1,17/08/2024,16:00,Brest,Marseille,1,5,A,1,3,...,1.89,2.04,1.91,2.02,1.91,2.08,1.85,2.01,1.90,2.09
2,F1,17/08/2024,18:00,Reims,Lille,0,2,A,0,1,...,2.10,1.70,2.14,1.81,2.14,1.85,2.07,1.80,2.13,1.86
3,F1,17/08/2024,20:00,Monaco,St Etienne,1,0,H,1,0,...,2.00,1.93,2.00,1.93,2.00,2.01,1.95,1.91,1.95,2.00
4,F1,18/08/2024,14:00,Auxerre,Nice,2,1,H,1,1,...,1.82,2.11,1.82,2.13,1.83,2.17,1.77,2.10,1.84,2.16


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Columns: 119 entries, Div to BFECAHA
dtypes: float64(96), int64(16), object(7)
memory usage: 284.6+ KB


# 1. Nettoyage et préparation des données

- Supprimer ou gérer les valeurs manquantes (NaN) pour les colonnes clés (scores, cotes, cartons, tirs)

In [11]:
df.isnull().sum()

Div         0
Date        0
Time        0
HomeTeam    0
AwayTeam    0
           ..
MaxCAHA     0
AvgCAHH     0
AvgCAHA     0
BFECAHH     0
BFECAHA     0
Length: 119, dtype: int64

- Standardiser les noms d’équipes (éviter doublons ou variantes)

In [16]:
sorted(set(df['HomeTeam']) | set(df['AwayTeam']))

['Angers',
 'Auxerre',
 'Brest',
 'Le Havre',
 'Lens',
 'Lille',
 'Lyon',
 'Marseille',
 'Monaco',
 'Montpellier',
 'Nantes',
 'Nice',
 'Paris SG',
 'Reims',
 'Rennes',
 'St Etienne',
 'Strasbourg',
 'Toulouse']

- Convertir les colonnes de date en datetime pour faciliter les filtrages par période

In [46]:
date = pd.to_datetime(df['Date'], format="%d/%m/%Y")
print(date)

0     2024-08-16
1     2024-08-17
2     2024-08-17
3     2024-08-17
4     2024-08-18
         ...    
301   2025-05-17
302   2025-05-17
303   2025-05-17
304   2025-05-17
305   2025-05-17
Name: Date, Length: 306, dtype: datetime64[ns]


- Créer des colonnes dérivées
    - GoalDiff = HomeGoals - AwayGoals
    - TotalGoals = HomeGoals + AwayGoals
    - MatchResultBinary (optionnel)

In [64]:
df["GoalDiff"] = df["FTHG"] - df["FTAG"]
df["TotalGoals"] = df["FTHG"] + df["FTAG"]
# df["AwayWin"] = np.where(df["FTR"] == "A", 1, 0)
# df.drop("NomDeLaColonne", axis=1
df.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA,GoalDiff,TotalGoals
0,F1,16/08/2024,19:45,Le Havre,Paris SG,1,4,A,0,1,...,1.95,1.97,1.95,2.02,1.89,1.95,1.95,2.02,-3,5
1,F1,17/08/2024,16:00,Brest,Marseille,1,5,A,1,3,...,1.91,2.02,1.91,2.08,1.85,2.01,1.90,2.09,-4,6
2,F1,17/08/2024,18:00,Reims,Lille,0,2,A,0,1,...,2.14,1.81,2.14,1.85,2.07,1.80,2.13,1.86,-2,2
3,F1,17/08/2024,20:00,Monaco,St Etienne,1,0,H,1,0,...,2.00,1.93,2.00,2.01,1.95,1.91,1.95,2.00,1,1
4,F1,18/08/2024,14:00,Auxerre,Nice,2,1,H,1,1,...,1.82,2.13,1.83,2.17,1.77,2.10,1.84,2.16,1,3


# 2. Analyse de performance dynamique

**Objectif :** suivre l’évolution des équipes au fil de la saison

- Calculer les points cumulés par journée pour chaque équipe (`cumsum`)

In [94]:
# Créer une colonne "Points" pour chaque match
df["PointsHome"] = np.where(df["FTR"] == "H", 3,
                   np.where(df["FTR"] == "D", 1, 0))

df["PointsAway"] = np.where(df["FTR"] == "A", 3,
                   np.where(df["FTR"] == "D", 1, 0))

# Création DataFrame Home & Away
dfHome = pd.DataFrame({
    "Team": df["HomeTeam"],
    "Points": df["PointsHome"],
    "MatchDay": date
})

dfAway = pd.DataFrame({
    "Team": df["AwayTeam"],
    "Points": df["PointsAway"],
    "MatchDay": date
})

# Concaténer les 2 DataFrame
dfAll = pd.concat([dfHome, dfAway], ignore_index=True)

# Trier par équipe et par date
dfAll.sort_values(["Team", "MatchDay"], inplace = True)

# Calculer les points cumulés
df["TotalPoints"] = dfAll.groupby(["Team"])["Points"].cumsum()


# OPTIONNEL
# Grouper par équipe et sommer les points
dfRanking = dfAll.groupby("Team")["Points"].sum().reset_index()

# Trier par points décroissants
dfRanking = dfRanking.sort_values("Points", ascending=False)
print(dfRanking)

           Team  Points
12     Paris SG      84
7     Marseille      65
8        Monaco      61
5         Lille      60
11         Nice      60
16   Strasbourg      57
6          Lyon      57
4          Lens      52
2         Brest      50
1       Auxerre      42
17     Toulouse      42
14       Rennes      41
0        Angers      36
10       Nantes      36
3      Le Havre      34
13        Reims      33
15   St Etienne      30
9   Montpellier      16


- Tracer l’évolution du classement avec un **line plot** interactif

- Visualiser les séries de victoires/défaites via un **heatmap** des performances par journée

# 3. Analyse des “big matches” et rivalités

**Objectif :** identifier les matchs clés de la saison

- Sélectionner les matchs entre équipes du top 5 ou top vs bottom

- Calculer les marges de victoire moyennes pour ces rencontres

- Comparer la variance des cotes pour ces matchs (bookmakers moins d’accord)

- Visualisation: scatter plot des scores vs cotes ou histogramme des marges

# 4. Analyse “fan insights”

**Objectif :** comprendre ce qu’un supporter ou analyste peut retirer des stats

- Taux de buts marqués après la mi-temps vs avant

- Cartons et fautes par match pour les équipes “agressives”

- Ratio tirs cadrés → buts pour identifier les équipes les plus efficaces

- Visualisation: radar chart par équipe ou bar plots comparatif

# 5. Exploration prédictive simplifiée

**Objectif :** mettre en évidence des patterns sans entraîner un modèle complexe

- Créer un score “favori probable” basé sur les cotes

- Vérifier si le favori “prévu” gagne réellement plus de 70 % du temps

- Visualiser la fiabilité des prédictions avec des **bar charts** ou **heatmaps**

# 6. Approche data storytelling

**Objectif :** raconter une histoire visuelle sur la saison

- “Qui a été la meilleure équipe à domicile vs extérieur ?” → bar chart

- “Les matchs les plus serrés” → scatter plot des scores

- “Les surprises de la saison” → points où la cote favori ne correspond pas au résultat

# 7. Manipulations Python typiques

- `groupby` par équipe ou période pour résumer les stats

- `merge` ou `concat` pour combiner domicile/extérieur si nécessaire

- Création de colonnes dérivées avec `apply` ou `np.where`

- Calcul de ratios et métriques : efficacité tirs → buts, cartons/match, over/under accuracy

# 8. Visualisations métier

- Classement final des équipes (bar chart)

- Bilan domicile vs extérieur (side-by-side bar chart)

- Heatmap de corrélation tirs / buts / cartons

- ROI d’une stratégie simple de pari (line chart ou bar chart)

- Comparaison probabilité implicite vs fréquence réelle (scatter plot + line)